## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(input_shape=(32,32,3),filters=32,
                             kernel_size=(3,3),
                             padding='same',
                             activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''

classifier.add(BatchNormalization())

#'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(filters=32,
                             kernel_size=(3,3),
                             padding='same',
                             activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

##超過兩個就要選categorical_crossentrophy

classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100
50000/50000 [==============================] - 7s 133us/step - loss: 1.2760 - acc: 0.5561
Epoch 2/100
50000/50000 [==============================] - 5s 93us/step - loss: 0.8196 - acc: 0.7102
Epoch 3/100
50000/50000 [==============================] - 5s 93us/step - loss: 0.5810 - acc: 0.7961
Epoch 4/100
50000/50000 [==============================] - 5s 93us/step - loss: 0.3915 - acc: 0.8620
Epoch 5/100
50000/50000 [==============================] - 5s 93us/step - loss: 0.2460 - acc: 0.9162
Epoch 6/100
50000/50000 [==============================] - 5s 94us/step - loss: 0.1750 - acc: 0.9394
Epoch 7/100
50000/50000 [==============================] - 5s 94us/step - loss: 0.1308 - acc: 0.9556
Epoch 8/100
50000/50000 [==============================] - 5s 95us/step - loss: 0.1120 - acc: 0.9608
Epoch 9/100
50000/50000 [==============================] - 5s 94us/step - loss: 0.1022 - 

50000/50000 [==============================] - 5s 92us/step - loss: 0.0174 - acc: 0.9948
Epoch 63/100
50000/50000 [==============================] - 5s 99us/step - loss: 0.0129 - acc: 0.9957
Epoch 64/100
50000/50000 [==============================] - 5s 99us/step - loss: 0.0105 - acc: 0.9964
Epoch 65/100
50000/50000 [==============================] - 5s 96us/step - loss: 0.0158 - acc: 0.9951
Epoch 66/100
50000/50000 [==============================] - 5s 94us/step - loss: 0.0226 - acc: 0.9929
Epoch 67/100
50000/50000 [==============================] - 5s 92us/step - loss: 0.0173 - acc: 0.9947
Epoch 68/100
50000/50000 [==============================] - 5s 91us/step - loss: 0.0163 - acc: 0.9952
Epoch 69/100
50000/50000 [==============================] - 5s 91us/step - loss: 0.0117 - acc: 0.9965
Epoch 70/100
50000/50000 [==============================] - 5s 92us/step - loss: 0.0168 - acc: 0.9950
Epoch 71/100
50000/50000 [==============================] - 5s 94us/step - loss: 0.0176 - acc: 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.3107541e-03, 5.7015936e-14, 3.3577983e-03, 9.9428248e-01,
        1.0490124e-03, 4.0066895e-12, 5.5692855e-09, 2.3729685e-10,
        4.7306808e-11, 2.3619612e-16]], dtype=float32)